Task1_0829. 가상 데이터 생성 (generate_data 함수) 후 모델링 및 평가하세요
- generate_data 함수는 1000개의 랜덤 시퀀스 데이터를 생성합니다.
  - vocab_size: 시퀀스에 사용할 어휘의 크기를 설정합니다. 여기서는 100개의 단어를 사용합니다.
  - data: 각 시퀀스는 seq_length=10으로 설정된 10개의 정수(단어 인덱스)로 구성됩니다.
  - labels: 각 시퀀스에 대해 0 또는 1의 이진 레이블을 무작위로 할당합니다.

- LSTM 기반 분류 모델을 정의하고, 가상 데이터로 학습 및 검증을 수행합니다.
- 조기 종료를 통해 학습 중 성능이 더 이상 개선되지 않을 때 학습을 중단합니다.
최종적으로 테스트 데이터로 최고 성능의 모델을 평가합니다.

#### nn.Embedding(vocag_size,embed_dim)
- 텍스트 데이터(LSTM 모델): 텍스트 데이터는 이산적인 정수로 표현되므로, 이 정수들을 고차원 벡터로 매핑하는 nn.Embedding 계층이 필요합니다. 이 임베딩 계층은 단어 간의 의미적 유사성을 학습하는 데 유용합니다.

- 이미지 데이터(CNN 모델): 이미지 데이터는 이미 공간적 구조를 가진 연속적인 값(픽셀)으로 표현되므로, 임베딩 계층이 필요하지 않습니다. 대신, 합성곱 계층이 이미지의 패턴을 학습하는 데 사용됩니다.

LSTM 모델의 경우 텍스트 데이터의 정수 인덱스를 벡터로 변환하기 위해 nn.Embedding이 필요하지만, CNN 모델에서는 이미지 데이터를 처리하는 데 이미 직접적인 합성곱 연산이 사용되므로 임베딩 계층이 필요하지 않습니다.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split

# 가상 데이터 생성 함수
def generate_data(num_samples=1000, seq_length=10):
    vocab_size = 100  # 어휘집 크기
    data = torch.randint(0, vocab_size, (num_samples, seq_length))
    labels = torch.randint(0, 2, (num_samples,))  # 0 또는 1의 레이블
    return data, labels

data, labels = generate_data()

# 강사님 버전

In [ ]:

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split

# 가상 데이터 생성 함수
def generate_data(num_samples=1000, seq_length=10):
    vocab_size = 100  # 어휘집 크기
    data = torch.randint(0, vocab_size, (num_samples, seq_length))
    labels = torch.randint(0, 2, (num_samples,))  # 0 또는 1의 레이블
    return data, labels

data, labels = generate_data()

# 텐서 데이터셋 및 데이터 로더 생성
dataset = TensorDataset(data, labels)
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - (train_size + val_size)
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# LSTM 모델 클래스 정의
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, output_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, text):
        embedded = self.embedding(text)
        lstm_out, (hidden, _) = self.lstm(embedded)
        hidden = hidden[-1,:,:]
        return self.fc(hidden)

model = LSTMModel(vocab_size=100, embed_dim=50, hidden_dim=100, output_dim=1)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters())

# 훈련 함수
def train(model, train_loader, optimizer, criterion):
    model.train()
    total_loss = 0
    for data, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(data).squeeze(1)
        loss = criterion(outputs, labels.float())
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)

# 평가 함수
def evaluate(model, data_loader, criterion):
    model.eval()
    total_loss = 0
    total_accuracy = 0
    with torch.no_grad():
        for data, labels in data_loader:
            outputs = model(data).squeeze(1)
            loss = criterion(outputs, labels.float())
            total_loss += loss.item()
            predictions = torch.round(torch.sigmoid(outputs))
            correct_predictions = (predictions == labels.unsqueeze(1)).float()
            total_accuracy += correct_predictions.sum().item()
    return total_loss / len(data_loader), total_accuracy / len(data_loader.dataset)

# 조기 종료 로직을 포함한 훈련 및 검증 과정
best_val_loss = float('inf') # float('inf')는 파이썬에서 양의 무한대를 나타내는 방식
patience = 3
trials = 0

for epoch in range(20):
    train_loss = train(model, train_loader, optimizer, criterion)
    val_loss, val_accuracy = evaluate(model, val_loader, criterion)
    print(f'Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        trials = 0
        torch.save(model.state_dict(), 'best_model.pth')
    else:
        trials += 1
        if trials >= patience:
            print("조기 종료 발생")
            break

# 테스트 데이터로 최고 모델 평가
model.load_state_dict(torch.load('best_model.pth'))
test_loss, test_accuracy = evaluate(model, test_loader, criterion)
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')


nn.BCEWithLogitsLoss는 PyTorch에서 이진 분류(Binary Classification) 작업을 수행할 때 주로 사용하는 손실 함수입니다. 이 함수는 이진 교차 엔트로피 손실(Binary Cross Entropy Loss)와 시그모이드(Sigmoid) 함수를 결합한 형태로 제공됩니다.